In [1]:
# Setup
! pip install seqeval evaluate
! pip install kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=1c5614bcd9cd71e187b22474826e8cdd26ea29c82374604112546d2082fd0921
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0

In [2]:
# Library imports
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoConfig, DistilBertForTokenClassification, DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.tokenization_utils_base import BatchEncoding
from datasets import Dataset, DatasetDict
import torch
import torch.nn as nn
from google.colab import drive, userdata
import pickle
import random
import re
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px
from seqeval.metrics import classification_report
import evaluate
import pprint
import kaleido

In [3]:
# Mount drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [4]:
# View all pandas columns, rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
metric = evaluate.load('seqeval')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [20]:
# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

# Define file write function
def write_pickle(dict_file, dict_file_name):
  dict_file_name = dict_file_name + '.pkl'
  with open(dict_file_name, 'wb') as file:
    pickle.dump(dict_file, file)

In [7]:
# Read in dictionary
all_data = read_pickle('all_data_matches.pkl')
all_data_amended = all_data.copy()

In [8]:
random_sample = random.sample(list(all_data_amended.items()), 3)

my_list = []
key, sub_dict = random_sample[0]
my_list.append(sub_dict['tokens'])
my_list.append(sub_dict['baseline_tags'])
my_list.append(sub_dict['input_ids'])
my_list.append(sub_dict['ner_tags'])
print(f"Key: {key}")
print(f"Text: {sub_dict['text']}")

pd.DataFrame(my_list, index=["Tokens", "Baseline_Tags", "Input_ids", "NER_Tags"])

Key: Cubic Number
Text: A cubic number is an integer that can be expressed as the cube of another integer. This means it is the result of multiplying a whole number by itself twice more. Cubic numbers can be visually represented as the volume of a cube with side lengths equal to the integer.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
Tokens,[CLS],A,cubic,number,is,an,integer,that,can,be,expressed,as,the,cube,of,another,integer,.,This,means,it,is,the,result,of,multi,##p,##lying,a,whole,number,by,itself,twice,more,.,C,##ubi,##c,numbers,can,be,visually,represented,as,the,volume,of,a,cube,with,side,lengths,equal,to,the,integer,.,[SEP]
Baseline_Tags,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-MISC,B-MISC,B-MISC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
Input_ids,101,138,12242,1295,1110,1126,18157,1115,1169,1129,4448,1112,1103,22034,1104,1330,18157,119,1188,2086,1122,1110,1103,1871,1104,4321,1643,15318,170,2006,1295,1118,2111,3059,1167,119,140,25422,1665,2849,1169,1129,19924,2533,1112,1103,3884,1104,170,22034,1114,1334,10707,4463,1106,1103,18157,119,102
NER_Tags,IGN,O,B-RECREATIONAL-MATHEMATICS,E-RECREATIONAL-MATHEMATICS,O,O,S-NUMBER-THEORY,O,O,O,O,O,O,S-RECREATIONAL-MATHEMATICS,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-NUMBER-THEORY,E-NUMBER-THEORY,O,O,O,O,O,S-ALGEBRA,O,O,S-NUMBER-THEORY,O,O,O,O,O,O,S-GEOMETRY,O,O,O,O,O,O,O,O,O,O,O,IGN


In [9]:
# Finetune definitions
# dict_file_name = 'train_definitions.pkl'
# train_definitions = read_pickle(dict_file_name)

# key, sub_dict = random_sample[0]
# # all_data_amended["Kobon Triangle"]['ner_tags'][68] = 'O'
# # all_data_amended["Kobon Triangle"]['ner_tags'][69] = 'O'

# # train_definitions.update({key: all_data_amended[key]})

# dict_file_name = 'train_definitions.pkl'
# with open(dict_file_name, 'wb') as file:
#   pickle.dump(train_definitions, file)

# # Train definitions has the hand-labelled definitions
# train_definitions = read_pickle(dict_file_name)
# train_definitions.keys()

## Class distribution by main tag

In [10]:
# Function to count the tag definition per tag-list
def returns_count_per_class(ner_tag_list):
  my_dict = {}
  for tag in ner_tag_list:
    if tag not in ['O', 'IGN']:
      tag = '-'.join(tag.split('-')[1:])
    if tag not in my_dict:
      my_dict[tag] = 1
    else:
      my_dict[tag] += 1
  sorted_data = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=True))
  return sorted_data

In [11]:
# Function to summarise main tags
def returns_main_tags(all_data_amended):
  name, count_list, no_tag_list = [], [], []

  for key, sub_dict in all_data_amended.items():
    my_list, my_list_i = [],[]
    to_count = sub_dict['ner_tags']
    count = returns_count_per_class(to_count)

    for i, (k, v) in enumerate(count.items()):
      if k in ['O', 'IGN']:
        continue
      else:
        my_list.append(k)
        my_list_i.append(key)
    try:
      count_list.append(my_list[0])
      name.append(my_list_i[0])
    except:
      no_tag_list.append(key)
      continue

  for_df = {'name': name, 'main_concept': count_list}
  df = pd.DataFrame(for_df)
  summary = df[['main_concept']].groupby('main_concept').value_counts().reset_index(drop=False)
  summary = summary.sort_values(by="count", ascending=False).reset_index(drop=True)
  return summary, df, no_tag_list

In [12]:
summary, label_df, no_tag_list = returns_main_tags(all_data_amended)

for tag in no_tag_list:
  try:
    del all_data_amended[tag]
  except:
    continue

assert len(all_data_amended) == label_df.shape[0], 'Error'

In [13]:
fig = px.bar(summary, x='main_concept', y='count', hover_data=['main_concept', 'count'], text='count', color='count', height=500, color_continuous_scale='Viridis', title="Distribution by primary classification label")
fig.update_coloraxes(showscale=False)
fig.show()

### Merging labels together

In [14]:
# Condense matrix classifications to single classification = "MATRICES"

b_tags = ["B-MATRIX-DECOMPOSITION", "B-MATRIX-EIGENVALUES", "B-MATRIX-GROUPS", "B-MATRIX-INVERSION", "B-MATRIX-NORMS", "B-MATRIX-OPERATIONS", "B-MATRIX-PROPERTIES", "B-MATRIX-TYPES", "B-INTEGER-MATRICES"]
i_tags = ["I-MATRIX-DECOMPOSITION","I-MATRIX-EIGENVALUES","I-MATRIX-GROUPS","I-MATRIX-INVERSION","I-MATRIX-NORMS","I-MATRIX-OPERATIONS","I-MATRIX-PROPERTIES","I-MATRIX-TYPES", "I-INTEGER-MATRICES"]
e_tags = ["E-MATRIX-DECOMPOSITION","E-MATRIX-EIGENVALUES","E-MATRIX-GROUPS","E-MATRIX-INVERSION","E-MATRIX-NORMS","E-MATRIX-OPERATIONS","E-MATRIX-PROPERTIES","E-MATRIX-TYPES", "E-INTEGER-MATRICES"]
s_tags = ["S-MATRIX-DECOMPOSITION","S-MATRIX-EIGENVALUES","S-MATRIX-GROUPS","S-MATRIX-INVERSION","S-MATRIX-NORMS","S-MATRIX-OPERATIONS","S-MATRIX-PROPERTIES","S-MATRIX-TYPES", "S-INTEGER-MATRICES"]
lie_b_tags = ["B-LIE-GROUPS", "B-LIE-THEORY"]
lie_i_tags = ["I-LIE-GROUPS", "I-LIE-THEORY"]
lie_e_tags = ["E-LIE-GROUPS", "E-LIE-THEORY"]
lie_s_tags = ["S-LIE-GROUPS", "S-LIE-THEORY"]

for i, (key, sub_dict) in enumerate(all_data_amended.items()):
  ner_tags = sub_dict['ner_tags']
  for ner_tag in ner_tags:
    if ner_tag in b_tags:
      ner_tags[ner_tags.index(ner_tag)] = "B-MATRICES"
    elif ner_tag in i_tags:
      ner_tags[ner_tags.index(ner_tag)] = "I-MATRICES"
    elif ner_tag in e_tags:
      ner_tags[ner_tags.index(ner_tag)] = "E-MATRICES"
    elif ner_tag in s_tags:
      ner_tags[ner_tags.index(ner_tag)] = "S-MATRICES"
    elif ner_tag in lie_b_tags:
      ner_tags[ner_tags.index(ner_tag)] = "B-LIE-ALGEBRA"
    elif ner_tag in lie_i_tags:
      ner_tags[ner_tags.index(ner_tag)] = "I-LIE-ALGEBRA"
    elif ner_tag in lie_e_tags:
      ner_tags[ner_tags.index(ner_tag)] = "E-LIE-ALGEBRA"
    elif ner_tag in lie_s_tags:
      ner_tags[ner_tags.index(ner_tag)] = "S-LIE-ALGEBRA"
  all_data_amended[key]['ner_tags'] = ner_tags

summary, label_df, no_tag_list = returns_main_tags(all_data_amended)
label_df = label_df.reset_index(drop=False)
fig = px.bar(summary, x='main_concept', y='count', hover_data=['main_concept', 'count'], text='count', color='count', height=500, color_continuous_scale='Viridis', title="Distribution by primary classification label")
fig.update_coloraxes(showscale=False)
fig.update_layout(autosize=False, width=1600, height=700, margin=dict( l=50, r=50, b=100, t=100, pad=4 ), font=dict(size=12))
fig.write_image("/content/drive/MyDrive/Colab Notebooks/Math_Graph/Images/fig1.png")
fig.show()

In [15]:
# Check order of label_df same as all_data_amended (NB for indices for train-test split)
for i, key in enumerate(list(all_data_amended.keys())):
  assert label_df.iloc[i]['name'] == key, 'Error'

label_df.head()

,index,name,main_concept
0,0,ludwig's inversion formula,CALCULUS-AND-ANALYSIS
1,1,quotient,NUMBER-THEORY
2,2,survivorship curve,APPLIED-MATHEMATICS
3,3,inadmissible,DISCRETE-MATHEMATICS
4,4,Natural Logarithm of 2,DISCRETE-MATHEMATICS


## Train-test stratified split

In [16]:
X_train_indices, X_test_indices, y_train_indices, y_test_indices = train_test_split(label_df['index'].to_numpy(), label_df['main_concept'].to_numpy(),
                                                                                    test_size=0.3, random_state=42, stratify=label_df['main_concept'].to_numpy())

# Repeat to get validation sub-sample of Train
X_train_indices, X_valid_indices, y_train_indices, y_valid_indices = train_test_split(X_train_indices, y_train_indices, test_size=0.3, random_state=42, stratify=y_train_indices)

In [17]:
# Check the resulting distributions
print("Class distribution in original dataset:")
print(label_df['main_concept'].value_counts(normalize=True))

print("\nClass distribution in train dataset:")
print(pd.Series(y_train_indices).value_counts(normalize=True))

print("\nClass distribution in validation dataset:")
print(pd.Series(y_valid_indices).value_counts(normalize=True))

Class distribution in original dataset:
main_concept
CALCULUS-AND-ANALYSIS          0.215153
NUMBER-THEORY                  0.145139
TOPOLOGY                       0.108790
ALGEBRA                        0.105596
GEOMETRY                       0.091159
DISCRETE-MATHEMATICS           0.087070
PROBABILITY-AND-STATISTICS     0.078510
APPLIED-MATHEMATICS            0.041204
MATRICES                       0.040437
RECREATIONAL-MATHEMATICS       0.035901
FOUNDATIONS-OF-MATHEMATICS     0.032196
LINEAR-ALGEBRA                 0.004855
LIE-ALGEBRA                    0.004088
ORG                            0.003322
DETERMINANTS                   0.003066
LOC                            0.002236
LINEAR-SYSTEMS-OF-EQUATIONS    0.001278
Name: proportion, dtype: float64

Class distribution in train dataset:
CALCULUS-AND-ANALYSIS          0.215152
NUMBER-THEORY                  0.145130
TOPOLOGY                       0.108750
ALGEBRA                        0.105620
GEOMETRY                       0.091

In [21]:
write_pickle(all_data_amended, 'all_data_amended')

In [22]:
indices = [X_train_indices, X_test_indices, y_train_indices, y_test_indices, X_valid_indices, y_valid_indices]
names = ['X_train_indices', 'X_test_indices', 'y_train_indices', 'y_test_indices', 'X_valid_indices', 'y_valid_indices']

for idx, name in zip(indices, names):
  write_pickle(idx, name)